In [1]:
import pandas as pd
expressionInThis = pd.read_csv('dataSample_rna_BRCA.csv',header = 0)
BRCA_index = pd.read_csv('BRCA_index.csv',header=0)
peakInthis = pd.read_csv('dataSample_atac_BRCA.csv',header = 0)

In [1]:
#这里有大问题啊啊啊啊我忘记求log2了
import pandas as pd
expressionInThis = pd.read_csv('dataSample_rna_BRCA_11.csv',header = 0)

In [2]:
columns = expressionInThis.columns.tolist()
import numpy as np 

for item in columns:
    expressionInThis[item] = np.log2(expressionInThis[item]+1)

In [3]:
expressionInThis.to_csv('dataSample_rna_BRCA_norm_11.csv',index = 0)

In [2]:
geneleft = expressionInThis.columns.tolist()
gene_anno = pd.read_csv('geneid_anno.csv',header=0)
gene_anno.index = gene_anno['loadedanno.gene_id']
chromosomes = BRCA_index['seqnames'].value_counts()

gene_anno

,chr,start,end,width,loadedanno.gene_id,loadedanno.type,exonlen
loadedanno.gene_id,,,,,,,
ENSG00000223972.5,chr1,11869,14409,2541,ENSG00000223972.5,gene,2289.0
ENSG00000227232.5,chr1,14404,29570,15167,ENSG00000227232.5,gene,1351.0
ENSG00000278267.1,chr1,17369,17436,68,ENSG00000278267.1,gene,68.0
ENSG00000243485.3,chr1,29554,31109,1556,ENSG00000243485.3,gene,1247.0
ENSG00000274890.1,chr1,30366,30503,138,ENSG00000274890.1,gene,138.0
...,...,...,...,...,...,...,...
ENSG00000224240.1,chrY,26549425,26549743,319,ENSG00000224240.1,gene,319.0
ENSG00000227629.1,chrY,26586642,26591601,4960,ENSG00000227629.1,gene,342.0
ENSG00000237917.1,chrY,26594851,26634652,39802,ENSG00000237917.1,gene,2337.0


In [4]:
gene_anno = gene_anno.loc[geneleft]
gene_anno

,chr,start,end,width,loadedanno.gene_id,loadedanno.type,exonlen
loadedanno.gene_id,,,,,,,
ENSG00000000003.13,chrX,100627109,100639991,12883,ENSG00000000003.13,gene,8747.0
ENSG00000000005.5,chrX,100584802,100599885,15084,ENSG00000000005.5,gene,1881.0
ENSG00000000419.11,chr20,50934867,50958555,23689,ENSG00000000419.11,gene,5867.0
ENSG00000000457.12,chr1,169849631,169894267,44637,ENSG00000000457.12,gene,15985.0
ENSG00000000460.15,chr1,169662007,169854080,192074,ENSG00000000460.15,gene,21871.0
...,...,...,...,...,...,...,...
ENSG00000281910.1,chr16,58559796,58559931,136,ENSG00000281910.1,gene,136.0
ENSG00000281912.1,chr1,45303910,45305619,1710,ENSG00000281912.1,gene,1710.0
ENSG00000281918.1,chr1,113079537,113079847,311,ENSG00000281918.1,gene,311.0


In [4]:
gene_anno = gene_anno.reset_index(drop=True)
gene_anno.sort_values(by=['chr','start'],axis=0,ascending=[True,True],inplace=True)
gene_anno.reset_index(drop =True, inplace = True)


In [5]:
#先只有一个 chr1的：
pearson_score = []
pearson_p = []
genename = []
peakname = []
chro = []
from scipy.stats import pearsonr
for j in range(0,len(chromosomes)):
    suoyin = chromosomes.index[j]
    gene_loc = gene_anno[gene_anno['chr']==suoyin]
    gene_loc.reset_index(drop =True, inplace = True)
    #对每个染色体都来一遍
    peak_loc = BRCA_index[BRCA_index['seqnames']==suoyin]
    peak_loc.reset_index(drop =True, inplace = True)
    # 写个算相关系数的
    #try1 = pearsonr(peak_loc.start,peak_loc.end) 
    initial_loc = 0 

    for i in range(len(gene_loc)):
        g_name = gene_loc['loadedanno.gene_id'][i]
        try:
            while gene_loc.start[i]-peak_loc.start[initial_loc] > 5000000:
                initial_loc = initial_loc+1
                if initial_loc == len(peak_loc):
                    break
            #print(str(initial_loc))
        #现在的话应该是大于-500了
        except:
            print('Error occur'+str(initial_loc)+str(len(peak_loc)))
        if initial_loc == len(peak_loc):
            continue
        target_loc = initial_loc
        while gene_loc.start[i]-peak_loc.start[target_loc] > -5000000:
            p_name = peak_loc.name[target_loc]
            c_result = pearsonr(expressionInThis[g_name],peakInthis[p_name])
            pearson_score.append(c_result[0])
            pearson_p.append(c_result[1])
            genename.append(g_name)
            peakname.append(p_name)
            chro.append(suoyin)
            target_loc = target_loc +1
            if target_loc == len(peak_loc):
                break
        #print(str(target_loc))
        
        #print(str(i))
    #print('finished_'+str(gene_loc.start[i]))    
    print('finished_'+ suoyin)

finished_chr22
finished_chrX
finished_chr21


In [6]:
output = pd.DataFrame({'chr': chro, 'geneid': genename,'peakname':peakname,'pearson':pearson_score,'p':pearson_p})
output


output.to_csv('test_pearson_onBRCA_2.csv',index=0)

In [7]:
from scipy.stats import pearsonr
import numpy as np
gene_name = []
variance = []
mean = []

peak_loc = BRCA_index
peak_loc.reset_index(drop =True, inplace = True)
randomsample = 10000
randomlist = np.random.rand(len(peak_loc))
random_sorted = np.sort(randomlist)
threshold = random_sorted[randomsample]
peak_loc['random'] = randomlist
peak_loc = peak_loc[peak_loc['random']<threshold]    
choosed = peak_loc['name'].tolist()
notfinded = 0

for j in range(0,len(chromosomes)):
    suoyin = chromosomes.index[j]
    gene_loc = gene_anno[gene_anno['chr']==suoyin]
    gene_loc.reset_index(drop =True, inplace = True)
    #对每个染色体都来一遍    
    for i in range(0,len(gene_loc)):
        theid = gene_loc['loadedanno.gene_id'][i]
        gene_name.append(theid)
        corlist = []
        try:
            expre = 
            for item in choosed:            
                cor = pearsonr(peakInthis[item],expressionInThis[theid])
                corlist.append(cor[0])
                vari = np.var(corlist)
                meani = np.mean(corlist)
                variance.append(vari)
                mean.append(meani)
            except:
                notfinded = notfinded+1
                print('fail_find_'+theid)

        
    print('finished_'+suoyin)
    
forFDR = pd.DataFrame({'gene_id':gene_name,'mean':mean,'var':variance})

forFDR.to_csv('forFDR.csv',index=0)
print('finished')

KeyboardInterrupt: 

In [9]:
#expressionInThis['ENSG00000278791.1']
gene_anno[gene_anno['loadedanno.gene_id']=='ENSG00000278791.1']


,chr,start,end,width,loadedanno.gene_id,loadedanno.type,exonlen
loadedanno.gene_id,,,,,,,
ENSG00000278791.1,chr1,632325,632413,89,ENSG00000278791.1,gene,89.0


,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSG00000281902.1,ENSG00000281903.1,ENSG00000281905.1,ENSG00000281906.1,ENSG00000281909.1,ENSG00000281910.1,ENSG00000281912.1,ENSG00000281918.1,ENSG00000281920.1,ENSG00000281921.1
0,55.437429,0.177544,172.758012,13.193387,10.398581,4.536391,11.722076,174.036578,17.890193,83.474958,...,2.783007,0.746083,0.000000,0.0,0.660002,0.000000,7.323703,0.000000,4.264285,0.0
1,62.171070,190.531779,36.640151,12.233092,2.102397,6.559576,97.180674,117.694952,17.792876,58.680436,...,0.000000,0.741694,0.000000,0.0,0.000000,0.000000,6.212801,0.000000,1.561812,0.0
2,73.717541,100.017024,40.147320,11.045569,2.245857,10.366928,117.048266,152.419334,28.327168,51.060737,...,1.580410,0.726316,0.000000,0.0,0.000000,0.000000,3.992615,0.000000,0.254905,0.0
3,28.174261,2.405006,29.630780,13.382038,6.421916,2.057191,23.100018,41.021370,25.728817,46.580448,...,0.000000,0.779178,6.398608,0.0,0.000000,38.015260,1.511718,0.692668,2.026799,0.0
4,44.389335,0.575517,79.002988,18.025519,8.637211,3.000391,11.352788,109.200490,21.961880,80.862888,...,1.503537,0.000000,0.000000,0.0,0.000000,0.000000,21.102280,0.000000,0.485012,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,35.055664,0.071687,53.022276,14.036782,7.645043,11.356211,40.816632,231.349229,27.685711,69.553206,...,2.247375,0.172139,0.000000,0.0,0.000000,0.000000,3.706198,0.433577,0.724960,0.0
85,53.175926,1.439895,44.348223,13.837313,3.434417,5.291872,36.898681,116.479679,14.818853,58.515423,...,0.000000,0.576264,0.000000,0.0,0.475791,0.000000,7.919422,0.580588,0.728076,0.0
86,30.350941,1.519423,16.941555,5.582371,1.088974,3.398594,10.839602,205.024994,5.130714,54.445392,...,2.646329,0.000000,0.000000,0.0,0.000000,2.334996,5.571218,19.400737,1.707309,0.0
87,84.798166,0.078802,42.772945,24.350639,7.543186,4.331992,23.636946,130.704869,12.186262,95.828928,...,0.000000,0.725368,0.000000,0.0,0.000000,0.000000,3.900719,0.000000,0.398461,0.0
